In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
base_url = "https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo"

# Páginas a recorrer
paginas = range(1, 60)  
all_links = []

for i in paginas:
    url = f"{base_url}/pagina-{i}"
    print(f"Scrapeando: {url}")
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")

    # Buscar tarjetas con la clase "card prop-entrada"
    tarjetas = soup.select("a.card.prop-entrada")

    # Extraer los hrefs
    links = [a['href'] for a in tarjetas if a.has_attr("href") and len(a['href']) > 10]
    links = list(set(links))  # eliminar duplicados

    all_links.extend(links)

# Normalizar los links (por si son relativos)
final_links = [f"https://www.casasymas.com.uy{link}" if link.startswith("/") else link for link in all_links]

# Mostrar resultados
print("\n✅ LINKS SCRAPEADOS:")
for link in final_links:
    print(link)
    
print(f"Total: {len(final_links)}")


Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-1
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-2
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-3
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-4
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-5
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-6
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-7
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-8
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-9
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-10
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/apartamento/montevideo/pagina-

KeyboardInterrupt: 

In [18]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

datos = []

for url in final_links:
    print(f"Procesando: {url}")
    try:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content, "html.parser")

        # Extraer precio
        precio = soup.select_one(".rate-info")
        precio = precio.get_text(strip=True) if precio else None

        # Extraer características
        caracteristicas = {}
        li_tags = soup.select("div.details-info li")
        
        for li in li_tags:
            img = li.find("img", class_="iconos")
            if img and img.has_attr("title"):
                key = img["title"].strip()
                text = li.get_text(strip=True)
                
                # Quitar el key del texto para obtener solo el valor
                value = text.replace(f"{key}:", "").strip()
                caracteristicas[key] = value

        # Guardar en lista
        datos.append({
            "url": url,
            "precio": precio,
            **caracteristicas
        })
        
        time.sleep(1)  # Pausa para no saturar el sitio

    except Exception as e:
        print(f"❌ Error en {url}: {e}")

# Convertir a DataFrame
df = pd.DataFrame(datos)
print(df.head())

# Guardar a CSV
df.to_csv("propiedades_scrapeadas.csv", index=False)
print("✅ Archivo guardado como propiedades_scrapeadas.csv")


Procesando: https://www.casasymas.com.uy/propiedad/142543-venta-de-loft-en-buceo-cosmos-home
Procesando: https://www.casasymas.com.uy/propiedad/173173-apartamento-en-venta-de-4-dormitorios-con-garage-en-pocitos-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/184349-apartamento-de-1-dormitorio-y-garaje-en-venta-con-renta-en-malvin
Procesando: https://www.casasymas.com.uy/propiedad/142544-venta-de-apartamento-1-dormitorio-en-buceo-cosmos-home
Procesando: https://www.casasymas.com.uy/propiedad/161378-apartamento-en-venta-de-3-dormitorios-con-garage-en-buceo-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/146925-venta-apartamento-1-dormitorio-buceo-casi-malvin-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/158489-apartamento-en-venta-de-3-dormitorios-en-golf-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/176139-apartamento-en-venta-de-3-dormitorios-con-garage-en-malvin-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/16

In [3]:
import pandas as pd

df_csv = pd.read_csv("propiedades_scrapeadas.csv")

In [5]:
df_csv.head(5)

,url,precio,Dormitorios,Baños,Superficie,Superficie Construida,Superficie Balcones,Cocina,Aptos por piso,Pisos Edificio,...,Lavadero,Orientación,Superficie Terreno,Altura,Patio,Dormitorios Servicio,Acepta Mascotas,Jardín,Plantas,Muebles
0,https://www.casasymas.com.uy/propiedad/142543-...,U$S 384.500Venta,Monoambiente,1.0,97 m²,77m²,97m²,Cocina Definida,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.casasymas.com.uy/propiedad/173173-...,U$S 410.000Venta,4,3.0,133 m²,NaN,NaN,Cocina Definida,NaN,NaN,...,Lavadero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.casasymas.com.uy/propiedad/184349-...,U$S 147.000Venta,1,1.0,45 m²,45m²,NaN,NaN,NaN,NaN,...,NaN,Norte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.casasymas.com.uy/propiedad/142544-...,U$S 258.500Venta,1,1.0,65 m²,47m²,65m²,Cocina Definida,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.casasymas.com.uy/propiedad/161378-...,U$S 183.000Venta,3,1.0,70 m²,70m²,NaN,Cocina Definida,2.0,NaN,...,Lavadero,NaN,70m²,3m,Patio,NaN,NaN,NaN,NaN,NaN
